In [1]:
import numpy as np
import fiona
from shapely.geometry import shape
import rasterio
from rasterio.windows import get_data_window
import matplotlib.pyplot as plt
from GISops import clip_raster

%matplotlib inline

### start with a 1 degree x 1 degree DEM tile downloaded from the national map
* this tile covers the La Crosse, WI area, from 43 to 44 deg. N, and 91 to 92 degrees W
* can be downloaded from https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/ArcGrid/n45w092.zip
* unzip and copy to the path below to run this script

In [ ]:
original_tile = 'data/n44w092/grdn44w092_13'

In [ ]:
with rasterio.open(original_tile) as src:
    plt.imshow(src.read(1))

In [ ]:
src.width, src.height # height and width in pixels

In [ ]:
src.bounds # bounding box in lat, lon

In [ ]:
src.res # x, y resolution in degrees

## Clip the raster using a window
#### read only the upper right quarter of the orignal tile and then write it back out
* use a window (https://mapbox.github.io/rasterio/topics/windowed-rw.html)
* the format for specifying the window is ((row_start, row_stop), (col_start, col_stop))
* note that the row/column numbering is in matrix order! (origin at upper left corner)

In [ ]:
window = ((0, src.height/4), (src.width/2, src.width))
window

In [ ]:
height = window[0][1] - window[0][0]
width = window[1][1] - window[1][0]
height, width

In [ ]:
with rasterio.open(original_tile) as src:
    # read in the upper right quarter
    img = src.read(1, window=window)
    
    # make a fresh copy of the meta data for the source raster
    kwargs = src.meta.copy()
    del kwargs['transform']
    
    # update the meta data with the new dimensions
    kwargs.update({
        'height': int(height),
        'width': int(width),
        'affine': src.window_transform(window), # set a new affine transform as well
        'driver': 'GTiff' # also switch the driver to a GeoTiff
    })
    
    # write it out!
    with rasterio.open('data/lc.tif', 'w', **kwargs) as dst:
         # since img is a 2-D array, specify that it should be written to band 1
        dst.write(img, 1)

In [ ]:
plt.imshow(img)

### reopen the file and verify that it was written correctly

In [ ]:
with rasterio.open('data/lc.tif') as src:
    plt.imshow(src.read(1))

## Clip the raster with an arbitrary polygon
* shapefile of counties
* this is also called masking (https://mapbox.github.io/rasterio/topics/masking-by-shapefile.html)
* [more info on Masks](https://github.com/mapbox/rasterio/blob/master/docs/topics/masks.rst) (this also bleeds into Mosaicing)

In [ ]:
counties = []
props = []
with fiona.open('data/counties.shp') as src:
    for record in src:
        counties.append(record['geometry'])
        props.append(record['properties'])
props

In [ ]:
counties[3] # GeoJSON representation of Houston County in MN

In [ ]:
from rasterio.tools.mask import mask

with rasterio.open('data/lc.tif') as src:
    out_image, out_transform = mask(src, [counties[3]], nodata=0, crop=True)
    out_meta = src.meta.copy()
    
    # also write this out as a GeoTiff
    # update the metadata to reflect the new shape of the masked image
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    with rasterio.open("data/Houston.tif", "w", **out_meta) as dest:
        dest.write(out_image)

In [ ]:
# above we set no data values to zero. 
#Mask them in the plot by converting them to nans
out_image[out_image == 0] = np.nan
plt.imshow(out_image[0, :, :])

### Clip another county, but this time use a buffer so that there's overap
(we will use the results in the Mosaic or Merge example to demonstrate handling of overlap)

#### convert the second county from GeoJSON to a shapely Polygon

In [ ]:
lc = shape(counties[0])
lc_buff = lc.simplify(0.03).buffer(0.04)
lc_buff.is_valid

#### inspect the buffer
* use some fancy unzipping/zipping to convert the coordinate sequence into lists of x and y locations, so we can make a plot with minimal code

In [ ]:
plt.plot(*zip(*lc.exterior.coords))
plt.plot(*zip(*lc_buff.exterior.coords))

#### go from Polygon back to GeoJSON

In [ ]:
lc_buff.__geo_interface__

In [ ]:
with rasterio.open('data/lc.tif') as src:
    out_image, out_transform = mask(src, [lc_buff.__geo_interface__], nodata=0, crop=True)
    out_meta = src.meta.copy()
    
    # also write this out as a GeoTiff
    # update the metadata to reflect the new shape of the masked image
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    with rasterio.open("data/LaCrosse.tif", "w", **out_meta) as dest:
        dest.write(out_image)

In [ ]:
out_image[out_image == 0] = np.nan
plt.imshow(out_image[0, :, :])